# **LIMPIEZA ENSAMBLE Y ANÁLISIS DE GENOMAS DE VIRUS DE INFLUENZA (PROGRAMA IRMA)**

## **PREPARAR AMBIENTE INICIAL**

Definir variables útiles, que corresponden al nombre de la muestra y a los nombres de los archivos fastq.gz

In [ ]:
MUESTRA=""
FASTQ1=""
FASTQ2=""
CONFIG_FLU_AVIAN="/home/admcenapa/analisis_influenza/bin/irma_config.sh"

Cambiar el directorio a la carpeta a procesar. Ya debe contener los archivos FASTQ a procesar. <span style="font-size: 150%; color: red;"> **Modificar dirección.**</span>

In [ ]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

## **ENSAMBLAJE POR IRMA**

Ejectuar el programa IRMA sobre los archivos crudos. Utilizar el módulo FLU-avian y el archivo de configuración prearmado (modificar si es necesario)
Como es una sola instrucción, sin bucles, puede mandarse a nohup.

In [ ]:
conda activate irma

nohup IRMA FLU-avian "$FASTQ1" "$FASTQ2" "./RESULTS" -c "$CONFIG_FLU_AVIAN" &

## **BLAST SOBRE RESULTADOS DE IRMA**

In [ ]:
cd "./RESULTS"

In [ ]:
conda activate ncbi
~/analisis_influenza/bin/irma_blast.sh

## **PRODUCIR FASTA FINAL**

In [ ]:
~/analisis_influenza/bin/irma_fasta_add_n.sh --table_blast blast_best_hit_hsp_sorted.tsv >"./${MUESTRA}.fna"

## **UNIR ARCHIVO DE PROFUNDIDADES**

In [ ]:
~/analisis_influenza/bin/irma_unir_prof.sh
mv prof.tsv ${MUESTRA}_prof.tsv

## **CORREGIR PROFUNDIDADES**

In [ ]:
conda activate R
~/analisis_influenza/bin/irma_corregir_prof.R --fasta ${MUESTRA}.fna --prof ${MUESTRA}_prof.tsv
conda deactivate

mv table_prof_corr.tsv ${MUESTRA}_prof_corr.tsv

## **GRAFICAR PROFUNDIDADES**

In [ ]:
conda activate R
Rscript ~/analisis_influenza/bin/irma_graficar_prof.R --input_file ${MUESTRA}_prof_corr.tsv --muestra ${MUESTRA}
Rscript ~/analisis_influenza/bin/irma_graficar_lecturas.R ./tables/READ_COUNTS.txt
mv plot_reads.png ${MUESTRA}_reads.png
conda deactivate

## **ANÁLISIS DE PATOGENICIDAD**

In [ ]:
conda activate base
~/analisis_influenza/bin/irma_analisis_pato.sh $(ls A_HA_*.fasta) >${MUESTRA}_pato.txt
conda deactivate

## **ANÁLISIS DE SUBTIPIFICACION**

In [ ]:
conda activate base
~/analisis_influenza/bin/irma_analisis_subtipificacion.sh ${MUESTRA}.fna
conda deactivate